In [1]:
from pyspark.sql import SparkSession
import utils

In [2]:
# start spark session
spark = SparkSession.builder.master("local[4]").appName("Restaurant Recommender").getOrCreate()

22/01/18 19:58:31 WARN Utils: Your hostname, Furkans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.33 instead (on interface en0)
22/01/18 19:58:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/18 19:58:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# get dataframes from csv files
ratings_df = spark.read.option("header", "true").csv("raw_data/rating_final.csv")
user_cuisine_df = spark.read.option("header", "true").csv("raw_data/usercuisine.csv")
user_payment_df = spark.read.option("header", "true").csv("raw_data/userpayment.csv")
user_profile_df = spark.read.option("header", "true").csv("raw_data/userprofile.csv")

In [4]:
# get selected columns from dataframes
ratings_df = ratings_df.selectExpr("userID as userId", "placeID as placeId", "rating as rating")
user_cuisine_df = user_cuisine_df.selectExpr("userID as userId", "Rcuisine as cuisine")
user_payment_df = user_payment_df.selectExpr("userID as userId", "Upayment as payment")
user_profile_df = user_profile_df.selectExpr("userID as userId", "smoker as smoker", "ambience as ambience", "transport as transport", "marital_status as marital_status", "activity as job", "budget as budget")

In [5]:
# merged data frames into one
merged_df = ratings_df.join(user_cuisine_df, ["userId"])
merged_df = merged_df.join(user_payment_df, ["userId"])
merged_df = merged_df.join(user_profile_df, ["userId"])

In [6]:
# unique user id list
unique_userid_list = []
unique_userid_list = merged_df.select('userId').distinct().collect()
len(unique_userid_list)

133

In [7]:
merged_df.groupBy('userId').count().show(1000)

+------+-----+
|userId|count|
+------+-----+
| U1012|   10|
| U1066|   10|
| U1042|    5|
| U1091|   18|
| U1019|    9|
| U1096|   11|
| U1014|   22|
| U1013|   20|
| U1125|   11|
| U1116|   33|
| U1105|    8|
| U1035|   12|
| U1069|   18|
| U1102|    5|
| U1101|  150|
| U1073|   11|
| U1049|    9|
| U1050|    7|
| U1134|   16|
| U1126|   10|
| U1007|    9|
| U1018|   10|
| U1067|    6|
| U1072|    8|
| U1031|    6|
| U1074|    6|
| U1059|   10|
| U1058|   10|
| U1055|   10|
| U1083|    9|
| U1033|   11|
| U1138|    3|
| U1061|   36|
| U1015|    8|
| U1131|    6|
| U1077|   15|
| U1108|  540|
| U1004|  144|
| U1029|   10|
| U1133|   18|
| U1001|    9|
| U1076|   15|
| U1027|   11|
| U1030|    8|
| U1021|   12|
| U1041|   24|
| U1127|    4|
| U1023|    4|
| U1047|    3|
| U1084|   10|
| U1062|   18|
| U1099|   22|
| U1060|   52|
| U1036|   12|
| U1114|   11|
| U1094|    8|
| U1020|    8|
| U1056|   10|
| U1115|    8|
| U1097|   13|
| U1017|    6|
| U1002|   10|
| U1006|   11|
| U1085|  

In [8]:
# unique place id list
unique_placeid_list = []
unique_placeid_list = merged_df.select('placeId').distinct().collect()
len(unique_placeid_list)

130

In [9]:
merged_df.groupBy('placeId').count().show(1000)

+-------+-----+
|placeId|count|
+-------+-----+
| 132834|  157|
| 132626|    4|
| 135058|  182|
| 135042|  139|
| 132767|   10|
| 135063|  118|
| 135026|   12|
| 132862|   39|
| 132884|    7|
| 135035|    5|
| 135088|   13|
| 135062|   54|
| 135021|   14|
| 135016|    5|
| 135072|    9|
| 135076|   16|
| 135051|   42|
| 132754|   20|
| 135054|   18|
| 132954|   31|
| 135050|    9|
| 132846|   14|
| 135044|    4|
| 135070|   10|
| 132717|   15|
| 132560|    4|
| 135069|   22|
| 135018|    7|
| 135034|    6|
| 135045|  131|
| 132885|    8|
| 135011|    6|
| 135060|  175|
| 135013|    6|
| 135052|  170|
| 132584|    8|
| 135001|   13|
| 132561|    4|
| 132613|    7|
| 132668|    7|
| 132866|   13|
| 135108|   20|
| 135019|   10|
| 132858|   15|
| 132660|    6|
| 132706|    4|
| 135028|  186|
| 135000|    7|
| 132830|   11|
| 135038|   51|
| 134996|   14|
| 135047|   13|
| 135048|    7|
| 132861|    8|
| 132665|    7|
| 132564|   16|
| 132583|    8|
| 132925|    8|
| 135027|   27|
| 134976

In [10]:
# unique cuisine types list
unique_cuisine_list = []
unique_cuisine_list = merged_df.select('cuisine').distinct().collect()
len(unique_cuisine_list)

103

In [11]:
merged_df.groupBy('cuisine').count().show(1000)

+--------------------+-----+
|             cuisine|count|
+--------------------+-----+
|       International|   14|
|           Cafeteria|  113|
|             Mexican|  999|
|        Cajun-Creole|   14|
|              Bakery|   34|
|                 Bar|   33|
|             Turkish|   26|
|        Scandinavian|   14|
|             Tibetan|   14|
|       North_African|   14|
|            Armenian|   14|
|             Dim_Sum|   14|
|      Middle_Eastern|   24|
|         Pacific_Rim|   14|
|             Persian|   14|
|   Dessert-Ice_Cream|   44|
|           Ethiopian|   14|
|               Swiss|   14|
|                Thai|   14|
|              Fusion|   27|
|               Sushi|   50|
|            Southern|   14|
|             Chinese|   71|
|          Indonesian|   14|
|             African|   14|
|           Mongolian|   24|
|     Southeast_Asian|   14|
|               Tapas|   14|
|            Hot_Dogs|   78|
|             Chilean|   14|
|               Irish|   14|
|           Fa

In [12]:
# unique payment types list
unique_payment_list = []
unique_payment_list = merged_df.select('payment').distinct().collect()
len(unique_payment_list)

5

In [13]:
merged_df.groupBy('payment').count().show()

+-------------------+-----+
|            payment|count|
+-------------------+-----+
|   bank_debit_cards|  217|
|               VISA|  300|
|MasterCard-Eurocard|  204|
|   American_Express|   17|
|               cash| 3352|
+-------------------+-----+



In [14]:
# unique smoker types list
unique_smoker_list = []
unique_smoker_list = merged_df.select('smoker').distinct().collect()
len(unique_smoker_list)

2

In [15]:
merged_df.groupBy('smoker').count().show()

+------+-----+
|smoker|count|
+------+-----+
| false| 3787|
|  true|  303|
+------+-----+



In [16]:
# unique ambience types list
unique_ambience_list = []
unique_ambience_list = merged_df.select('ambience').distinct().collect()
len(unique_ambience_list)

4

In [17]:
merged_df.groupBy('ambience').count().show()

+--------+-----+
|ambience|count|
+--------+-----+
| friends|  882|
|  family| 2427|
|       ?|   27|
|solitary|  754|
+--------+-----+



In [18]:
# unique transport types list
unique_transport_list = []
unique_transport_list = merged_df.select('transport').distinct().collect()
len(unique_transport_list)

4

In [19]:
merged_df.groupBy('transport').count().show()

+---------+-----+
|transport|count|
+---------+-----+
|   public| 1627|
|        ?|   26|
|car owner|  634|
|  on foot| 1803|
+---------+-----+



In [20]:
# unique marital status types list
unique_marital_status_list = []
unique_marital_status_list = merged_df.select('marital_status').distinct().collect()
len(unique_marital_status_list)

4

In [21]:
merged_df.groupBy('marital_status').count().show()

+--------------+-----+
|marital_status|count|
+--------------+-----+
|         widow|   31|
|       married|  131|
|        single| 3919|
|             ?|    9|
+--------------+-----+



In [22]:
# unique job types list 
unique_job_list = []
unique_job_list = merged_df.select('job').distinct().collect()
len(unique_job_list)

5

In [23]:
merged_df.groupBy('job').count().show()

+-------------+-----+
|          job|count|
+-------------+-----+
|working-class|    8|
|      student| 3655|
| professional|  385|
|            ?|   25|
|   unemployed|   17|
+-------------+-----+



In [24]:
# unique budget types list
unique_budget_list = []
unique_budget_list = merged_df.select('budget').distinct().collect()
len(unique_budget_list)

4

In [25]:
merged_df.groupBy('budget').count().show()

+------+-----+
|budget|count|
+------+-----+
|   low| 2012|
|  high|   86|
|medium| 1954|
|     ?|   38|
+------+-----+



In [26]:
merged_df.toPandas()

,userId,placeId,rating,cuisine,payment,smoker,ambience,transport,marital_status,job,budget
0,U1077,135085,2,Mexican,bank_debit_cards,false,family,public,married,student,medium
1,U1077,135085,2,Mexican,cash,false,family,public,married,student,medium
2,U1077,135085,2,Mexican,VISA,false,family,public,married,student,medium
3,U1077,135038,2,Mexican,bank_debit_cards,false,family,public,married,student,medium
4,U1077,135038,2,Mexican,cash,false,family,public,married,student,medium
...,...,...,...,...,...,...,...,...,...,...,...
4085,U1043,132630,1,Latin_American,cash,false,solitary,public,single,student,medium
4086,U1011,132715,1,Mexican,cash,false,family,public,single,student,medium
4087,U1068,132733,1,Mexican,cash,false,friends,public,single,student,low
4088,U1068,132594,1,Mexican,cash,false,friends,public,single,student,low


In [27]:
utils.create_new_df(unique_cuisine_list, unique_payment_list, unique_smoker_list, unique_ambience_list, unique_transport_list, unique_marital_status_list, unique_job_list, unique_budget_list)
new_df = spark.read.option("header", "true").csv("my_restaurant_dataset4.csv")
new_df = new_df.selectExpr("userId", "placeId", "rating", "cuisine", "payment", "smoker", "ambience", "transport", "marital_status", "job", "budget")
new_df.toPandas()

,userId,placeId,rating,cuisine,payment,smoker,ambience,transport,marital_status,job,budget
0,1.0,1.0,3.0,Italian,cash,false,family,public,single,student,low
1,2.0,2.0,1.0,Moroccan,cash,false,solitary,on foot,single,student,medium
2,3.0,3.0,5.0,Pizzeria,cash,false,friends,car owner,single,student,low
3,4.0,4.0,5.0,Middle_Eastern,cash,false,family,on foot,single,student,medium
4,5.0,5.0,4.0,Regional,cash,true,family,on foot,single,student,low
...,...,...,...,...,...,...,...,...,...,...,...
199995,1000.0,983.0,4.0,Korean,cash,false,friends,on foot,single,student,low
199996,1000.0,704.0,1.0,Mongolian,cash,false,family,on foot,single,student,low
199997,1000.0,870.0,1.0,Mexican,VISA,false,friends,public,single,student,low
199998,1000.0,468.0,1.0,Bagels,cash,false,friends,public,single,student,low


In [38]:
#new_df.groupBy('userId').count().show(1000)

In [39]:
#new_df.groupBy('placeId').count().show(1000)

In [30]:
new_df.groupBy('cuisine').count().show(1000)

+--------------------+-----+
|             cuisine|count|
+--------------------+-----+
|       International| 2145|
|           Cafeteria| 3364|
|             Mexican|52034|
|        Cajun-Creole| 1118|
|              Bakery|  567|
|                 Bar| 1837|
|             Tibetan| 1028|
|             Turkish| 2093|
|        Scandinavian|  683|
|       North_African|  115|
|            Armenian|  832|
|             Dim_Sum|   47|
|      Middle_Eastern|  246|
|         Pacific_Rim| 1396|
|             Persian|   39|
|   Dessert-Ice_Cream| 1177|
|           Ethiopian|  295|
|               Swiss|  456|
|                Thai| 1070|
|              Fusion|  354|
|            Southern|  167|
|               Sushi| 2536|
|             Chinese| 2516|
|          Indonesian|   43|
|             African|  431|
|     Southeast_Asian|  598|
|           Mongolian|  953|
|            Hot_Dogs| 3845|
|             Chilean| 1401|
|               Irish| 1208|
|           Fast_Food| 1988|
|             

In [31]:
new_df.groupBy('payment').count().show()

+-------------------+------+
|            payment| count|
+-------------------+------+
|   bank_debit_cards| 12097|
|               VISA| 16037|
|MasterCard-Eurocard| 10529|
|   American_Express|   388|
|               cash|160949|
+-------------------+------+



In [32]:
new_df.groupBy('smoker').count().show()

+------+------+
|smoker| count|
+------+------+
| false|185213|
|  true| 14787|
+------+------+



In [33]:
new_df.groupBy('ambience').count().show()

+--------+------+
|ambience| count|
+--------+------+
| friends| 43216|
|  family|119482|
|solitary| 37302|
+--------+------+



In [34]:
new_df.groupBy('transport').count().show()

+---------+-----+
|transport|count|
+---------+-----+
|   public|80108|
|car owner|31372|
|  on foot|88520|
+---------+-----+



In [35]:
new_df.groupBy('marital_status').count().show()

+--------------+------+
|marital_status| count|
+--------------+------+
|         widow|   656|
|       married|  6683|
|        single|192661|
+--------------+------+



In [36]:
new_df.groupBy('job').count().show()

+-------------+------+
|          job| count|
+-------------+------+
|working-class|  1117|
|      student|178910|
| professional| 19410|
|   unemployed|   563|
+-------------+------+



In [37]:
new_df.groupBy('budget').count().show()

+------+------+
|budget| count|
+------+------+
|   low|103578|
|  high|  4121|
|medium| 92301|
+------+------+

